In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Dense, Activation, Add
from keras.datasets import cifar10
from keras.utils import to_categorical

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [4]:
InitSeed = 767
tf.random.set_seed(InitSeed)
np.random.seed(InitSeed)

In [18]:
def ResidualUnit(x, filters, kn, str):
    tmp = x
    x = Conv2D(filters, kernel_size=kn, strides=str, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, kernel_size=kn, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    if str != 1 or x.shape[-1] != tmp.shape[-1]:
        tmp = Conv2D(filters, kernel_size=1, strides=str, padding='same')(tmp)
        tmp = BatchNormalization()(tmp)
    x = Add()([x, tmp])
    x = Activation('relu')(x)
    return x

In [20]:
input = Input(shape=(32, 32, 3))
x = Conv2D(64, kernel_size=7, strides=2, padding='same')(input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
indx = 0
for i in [64,64,64,128,128,128,256,256,256,512,512,512]:
    if indx == 3 or indx == 3 or indx == 3:
        x = ResidualUnit(x, i, 3, 2)
    else:
        x = ResidualUnit(x, i, 3, 1)
    indx+=1
x = GlobalAveragePooling2D()(x)
output = Dense(10, activation='softmax')(x)
ResNet50 = Model(inputs=input, outputs=output)

In [7]:
KerasModel = tf.keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

In [8]:
for layer in KerasModel.layers[:-1]:
    layer.trainable = False

In [9]:
x = KerasModel.output
x = GlobalAveragePooling2D()(x)
output = Dense(10, activation='softmax')(x)
model = Model(inputs=KerasModel.input, outputs=output)

In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
1563/1563 [==============================] - 231s 146ms/step - loss: 2.0758 - accuracy: 0.2423 - val_loss: 1.9060 - val_accuracy: 0.3196
Epoch 2/5
1563/1563 [==============================] - 219s 140ms/step - loss: 1.9180 - accuracy: 0.3090 - val_loss: 1.8983 - val_accuracy: 0.3179
Epoch 3/5
1563/1563 [==============================] - 215s 138ms/step - loss: 1.8602 - accuracy: 0.3354 - val_loss: 1.8261 - val_accuracy: 0.3486
Epoch 4/5
1563/1563 [==============================] - 214s 137ms/step - loss: 1.8280 - accuracy: 0.3475 - val_loss: 1.8020 - val_accuracy: 0.3537
Epoch 5/5
1563/1563 [==============================] - 217s 139ms/step - loss: 1.8051 - accuracy: 0.3571 - val_loss: 1.7577 - val_accuracy: 0.3790


In [15]:
loss, accuracy = model.evaluate(x_test, y_test)
accuracy

313/313 [==============================] - 37s 111ms/step - loss: 1.7577 - accuracy: 0.3790


0.3790000081062317